In [3]:

import json
import pandas as pd
import os

from google import genai
from google.genai import types
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM

In [13]:
model_google = "gemini-2.0-flash-lite" # "gemini-2.0-flash" "gemini-2.5-flash"
model_openai = "gpt-5-nano"  # "gpt-5" "gpt-4-turbo"
model_huggingface = "/Users/rodrigocarrillo/Documents/LLMs Hugging Face/gemma-3-4b-it"

In [5]:
def find_latest_file_simple(folder_path, pattern = 'rotafono_articles_scraped_'):
    files = [f for f in os.listdir(folder_path) 
             if f.startswith(pattern) and f.endswith('.pkl')]
    if not files:
        return None
    # Sort by filename (works because YYYYMMDD format sorts correctly)
    return os.path.join(folder_path, sorted(files)[-1])

In [6]:
df_with_scraped_data = find_latest_file_simple("/Users/rodrigocarrillo/Documents/Natural Language Processing Projects/Rotafono Scrape/01_Data_Text/")
articles_df = pd.read_pickle(df_with_scraped_data)
articles_df = articles_df.sample(n=2, random_state=42).reset_index(drop=True)
articles_df

,title,url,page,category,publish_date,article_content
0,Chaclacayo: reportan basura acumulada en el pa...,https://rotafono.pe/casos/lima-chaclacayo-chac...,3,Servicios públicos,Publicado el 16-10-25,Servicios públicos\n\n¿Qué pasó?\n\nRotafono d...
1,Villa El Salvador: adolescente de 14 años desa...,https://rotafono.pe/casos/lima-villa-el-salvad...,8,Servicios a la comunidad,Publicado el 13-07-25,Servicios a la comunidad\n\nEl adolescente se ...


In [7]:
with open('/Users/rodrigocarrillo/Documents/Natural Language Processing Projects/Suggested Reviewers Emails/00.Code/config.json', 'r') as config_file:
    api_keys = json.load(config_file)
GOOGLE_API_KEY = api_keys['GOOGLE_API_KEY']
OPENAI_API_KEY = api_keys['OPENAI_API_KEY']

In [ ]:
# client = genai.Client(api_key=GOOGLE_API_KEY)

In [8]:
system_instruction = """

Rol: Eres un usuario experto de redes sociales.
Tarea: Tu tarea es crear publicaciones atractivas para una red social (Nextdoor) basadas en el contenido proporcionado.

Consideraciones clave:
1. Escribe en primera persona del singular, como si fueras un residente local compartiendo información con tus vecinos.
2. Enfócate en aspectos que interesen a la comunidad local, como eventos, noticias o recomendaciones.
3. Mantén un tono positivo y alentador para fomentar la participación de los vecinos.
4. Asegúrate de que las publicaciones sean concisas (<100 palabras), atractivas y adecuadas para la red social Nextdoor.
5. Solo devuelve el texto de la publicación sin explicaciones adicionales.
6. No incluyas numeros de telefono o enlaces web.
7. Utiliza un tono amigable y conversacional que invite a la interacción de los usuarios.

"""

In [ ]:
class SocialMediaPostGenerator:
    def __init__(self, model_huggingface_path):

        # Load all models once during initialization
        self.google_client = genai.Client(api_key=GOOGLE_API_KEY)
        self.openai_client = OpenAI(api_key=OPENAI_API_KEY)
        
        # Load HuggingFace model
        self.hf_tokenizer = None
        self.hf_model = None
        self.model_huggingface_path = model_huggingface_path
        self._load_huggingface_model()
    
    def _load_huggingface_model(self):
        """Load HuggingFace model"""
        print(f"Searching for model in: {self.model_huggingface_path}")
        
        # Check if path exists
        if not os.path.exists(self.model_huggingface_path):
            print(f"ERROR: Path does not exist: {self.model_huggingface_path}")
            return
        
        model_path = None
        for root, dirs, files in os.walk(self.model_huggingface_path):
            if 'tokenizer.model' in files or 'config.json' in files:
                model_path = root
                print(f"Found model files in: {model_path}")
                break
        
        if model_path is None:
            print(f"ERROR: No model files found in {self.model_huggingface_path}")
            print("Looking for 'tokenizer.model' or 'config.json'")
            return
        
        try:
            print("Loading tokenizer...")
            self.hf_tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
            print("Loading model...")
            self.hf_model = AutoModelForCausalLM.from_pretrained(model_path)
            print("HuggingFace model loaded successfully!")
        except Exception as e:
            print(f"ERROR loading model: {e}")
    
    def generate(self, article_content, provider='google'):
        if provider == 'google':
            response = self.google_client.models.generate_content(
                model=model_google,
                config=types.GenerateContentConfig(
                    system_instruction=system_instruction,
                    temperature=0.1
                ),
                contents=article_content
            )
            return response.text
        
        elif provider == 'huggingface':
            # Check if model loaded
            if self.hf_tokenizer is None or self.hf_model is None:
                return "ERROR: HuggingFace model not loaded"
            
            inputs = self.hf_tokenizer.apply_chat_template(
                [
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": article_content},
                ],
                add_generation_prompt=True,
                tokenize=True,
                return_dict=True,
                return_tensors="pt",
            ).to(self.hf_model.device)
            
            outputs = self.hf_model.generate(
                **inputs,
                max_new_tokens=400,
                temperature=0.1,
                do_sample=True,
                eos_token_id=self.hf_tokenizer.convert_tokens_to_ids("<end_of_turn>")
            )
            
            text = self.hf_tokenizer.decode(
                outputs[0][inputs["input_ids"].shape[-1]:],
                skip_special_tokens=True
            )
            return text.replace("<end_of_turn>", "").strip()
        
        elif provider == 'openai':
            response = self.openai_client.responses.create(
                model=model_openai,
                reasoning={"effort": "low"},
                input=[
                    {"role": "developer", "content": system_instruction},
                    {"role": "user", "content": article_content}
                ]
            )
            return response.output_text
        
        else:
            raise ValueError("Unsupported provider.")



In [ ]:
# def generate_social_media_post(article_content, provider: str = 'google'):
    
#     if provider == 'google':
#         google_client = genai.Client(api_key=GOOGLE_API_KEY)
#         response = google_client.models.generate_content(
#             model = model_google,
#             config = types.GenerateContentConfig(
#                 system_instruction = system_instruction,
#                 temperature = 0.1),
#             contents = article_content
#         )
#         return response.text
    
#     elif provider == 'openai':
#         openai_client = OpenAI(api_key = OPENAI_API_KEY)
#         response = openai_client.responses.create(
#             model = model_openai,
#             reasoning = {"effort": "low"},
#             input=[
#                 {
#                     "role": "developer",
#                     "content": system_instruction
#                 },
#                 {
#                     "role": "user",
#                     "content": article_content
#                 }
#             ]
#         )
#         return response.output_text
#     else:
#         raise ValueError("Unsupported provider. Choose 'google' or 'openai'.")


In [15]:
# articles_df['social_media_post'] = articles_df.apply(lambda row: generate_social_media_post(row['article_content'], provider='huggingface'), axis=1)
# Usage - pass the path explicitly
generator = SocialMediaPostGenerator(model_huggingface_path=model_huggingface)

# Check if model loaded before using
if generator.hf_model is not None:
    articles_df['social_media_post'] = articles_df.apply(
        lambda row: generator.generate(row['article_content'], provider='huggingface'),
        axis=1
    )
else:
    print("Cannot generate posts - model failed to load")

Searching for model in: /Users/rodrigocarrillo/Documents/LLMs Hugging Face/gemma-3-4b-it
Found model files in: /Users/rodrigocarrillo/Documents/LLMs Hugging Face/gemma-3-4b-it/models--google--gemma-3-4b-it/snapshots/093f9f388b31de276ce2de164bdc2081324b9767
Loading tokenizer...
Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✓ HuggingFace model loaded successfully!


In [16]:
articles_df

,title,url,page,category,publish_date,article_content,social_media_post
0,Chaclacayo: reportan basura acumulada en el pa...,https://rotafono.pe/casos/lima-chaclacayo-chac...,3,Servicios públicos,Publicado el 16-10-25,Servicios públicos\n\n¿Qué pasó?\n\nRotafono d...,¡Hola vecinos de Chaclacayo!\n\nMe preocupa ve...
1,Villa El Salvador: adolescente de 14 años desa...,https://rotafono.pe/casos/lima-villa-el-salvad...,8,Servicios a la comunidad,Publicado el 13-07-25,Servicios a la comunidad\n\nEl adolescente se ...,¡Hola vecinos!\n\nMe preocupa mucho la desapar...


In [ ]:
def generate_social_media_post(article_content):
    response = client.models.generate_content(
        model = model_google,
        config = types.GenerateContentConfig(
            system_instruction = system_instruction,
            temperature = 0.1),
        contents = article_content
    )
    return response.text

In [ ]:
articles_df[['response']] = articles_df.apply(lambda row: generate_social_media_post(row['article_content']), axis=1)

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.responses.create(
    model = "gpt-5-nano",
    reasoning = {"effort": "low"},
    input=[
        {
            "role": "developer",
            "content": system_instruction
        },
        {
            "role": "user",
            "content": articles_df['article_content'].iloc[0]
        }
    ]
)

print(response.output_text)

In [ ]:
response = client.models.generate_content(
    model = modelo,
    config = types.GenerateContentConfig(
        system_instruction = system_instruction,
        temperature = 0.1),
    contents = articles_df['article_content'].iloc[0]
)

In [ ]:
from IPython.display import display, Markdown
display(Markdown(response.text))


In [ ]:
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Check what's actually in your directories
import os

model_huggingface = '/Users/rodrigocarrillo/Documents/LLMs Hugging Face/gemma-3-4b-it'
for root, dirs, files in os.walk(model_huggingface):
    if 'tokenizer.model' in files or 'config.json' in files:
        print(f"Model files found in: {root}")
        tokenizer = AutoTokenizer.from_pretrained(root, use_fast=False)
        model = AutoModelForCausalLM.from_pretrained(root)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="/Users/rodrigocarrillo/Documents/LLMs Hugging Face/gemma-3-1b-it/models--google--gemma-3-1b-it/snapshots/dcc83ea841ab6100d6b47a070329e1ba4cf78752", use_fast=False)
# model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="/Users/rodrigocarrillo/Documents/LLMs Hugging Face/gemma-3-1b-it/models--google--gemma-3-1b-it/snapshots/dcc83ea841ab6100d6b47a070329e1ba4cf78752", device_map="auto")


In [ ]:
messages = [
    {"role": "system", "content": system_instruction},
    {"role": "user", "content": articles_df['article_content'].iloc[0]},
]

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=400, temperature=0.1, do_sample=True, eos_token_id=tokenizer.convert_tokens_to_ids("<end_of_turn>"))
text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
text = text.replace("<end_of_turn>", "").strip()
print(text)

In [ ]:
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True))